In [2]:
try:
    from ultralytics import YOLO
    print("Ultralytics sudah ada ✅")
except:
    print("Install ultralytics dulu ⏳")
    !pip install ultralytics


Install ultralytics dulu ⏳
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.4 MB/s eta 0:00:0000:01


In [3]:
import json
import os
import shutil
from PIL import Image

IMAGE_SRC_DIR = "/kaggle/input/balls-detection/oi5_ball"
JSON_PATH = "/kaggle/input/balls-detection/oi5_ball_filename_to_bbox_train.json"
OUT_BASE = "/kaggle/working/dataset"
IMG_OUT = os.path.join(OUT_BASE, "images/train")
LBL_OUT = os.path.join(OUT_BASE, "labels/train")

os.makedirs(IMG_OUT, exist_ok=True)
os.makedirs(LBL_OUT, exist_ok=True)

with open(JSON_PATH, "r") as f:
    data = json.load(f)

print("total image: ", len(data))

# convert

for img_rel_path, bboxes in data.items():
    img_name = os.path.basename(img_rel_path)
    img_src = os.path.join(IMAGE_SRC_DIR, img_name)

    if not os.path.exists(img_src):
        continue

    img = Image.open(img_src)
    w, h = img.size

    label_lines = []
    
    for box in bboxes:
        x_center, y_center, bw_norm, bh_norm = box
    
        label_lines.append(
            f"0 {x_center:.6f} {y_center:.6f} {bw_norm:.6f} {bh_norm:.6f}"
        )

    # save label
    label_path = os.path.join(LBL_OUT, img_name.replace(".jpg", ".txt"))
    with open(label_path, "w") as f:
        f.write("\n".join(label_lines))

    # copy image
    shutil.copy(img_src, os.path.join(IMG_OUT, img_name))

print("konversi selesai")

total image:  2875
konversi selesai


In [4]:
import os
import random
import shutil

BASE = "/kaggle/working/dataset"
IMG_TRAIN = f"{BASE}/images/train"
LBL_TRAIN = f"{BASE}/labels/train"

IMG_VAL = f"{BASE}/images/val"
LBL_VAL = f"{BASE}/labels/val"

os.makedirs(IMG_VAL, exist_ok=True)
os.makedirs(LBL_VAL, exist_ok=True)
os.makedirs("/kaggle/working/images/val", exist_ok=True)
os.makedirs("/kaggle/working/labels/val", exist_ok=True)
os.makedirs("/kaggle/working/images/train", exist_ok=True)
os.makedirs("/kaggle/working/labels/train", exist_ok=True)

imgs = os.listdir(IMG_TRAIN)
random.shuffle(imgs)

val_count = int(0.1 * len(imgs))
val_imgs = imgs[:val_count]

for img in val_imgs:
    shutil.move(f"{IMG_TRAIN}/{img}", f"{IMG_VAL}/{img}")
    shutil.move(
        f"{LBL_TRAIN}/{img.replace('.jpg','.txt')}",
        f"{LBL_VAL}/{img.replace('.jpg','.txt')}"
    )

print(f"Val images: {len(val_imgs)}")


Val images: 287


In [5]:
import os, shutil, random
from glob import glob

SRC_IMG = "/kaggle/input/balls-detection/oi5_ball"
SRC_LBL = "/kaggle/working/dataset/labels/train"

DST_IMG_TRAIN = "/kaggle/working/images/train"
DST_IMG_VAL   = "/kaggle/working/images/val"
DST_LBL_TRAIN = "/kaggle/working/labels/train"
DST_LBL_VAL   = "/kaggle/working/labels/val"

os.makedirs(DST_IMG_TRAIN, exist_ok=True)
os.makedirs(DST_IMG_VAL, exist_ok=True)
os.makedirs(DST_LBL_TRAIN, exist_ok=True)
os.makedirs(DST_LBL_VAL, exist_ok=True)

labels = glob(f"{SRC_LBL}/*.txt")
random.shuffle(labels)

val_ratio = 0.1
val_count = int(len(labels) * val_ratio)

for i, lbl in enumerate(labels):
    name = os.path.basename(lbl).replace(".txt", ".jpg")
    img_path = f"{SRC_IMG}/{name}"

    if not os.path.exists(img_path):
        continue

    if i < val_count:
        shutil.copy(img_path, f"{DST_IMG_VAL}/{name}")
        shutil.copy(lbl, f"{DST_LBL_VAL}/{name.replace('.jpg','.txt')}")
    else:
        shutil.copy(img_path, f"{DST_IMG_TRAIN}/{name}")
        shutil.copy(lbl, f"{DST_LBL_TRAIN}/{name.replace('.jpg','.txt')}")

print("dataset filtered & rebuilt CLEAN 🧼🔥")

!ls /kaggle/working/images/train | wc -l
!ls /kaggle/working/images/val | wc -l
!ls /kaggle/working/labels/train | wc -l
!ls /kaggle/working/labels/val | wc -l

dataset filtered & rebuilt CLEAN 🧼🔥
2330
258
2330
258


In [6]:
import shutil
import random
from glob import glob

train_imgs = glob("/kaggle/working/images/train/*.jpg")
random.shuffle(train_imgs)

val_count = int(0.1 * len(train_imgs))
val_imgs = train_imgs[:val_count]

for img_path in val_imgs:
    name = os.path.basename(img_path)

    # pindah img
    shutil.move(
        img_path,
        f"/kaggle/working/images/val/{name}"
    )

    # pundah label
    shutil.move(
        f"/kaggle/working/labels/train/{name.replace('.jpg','.txt')}",
        f"/kaggle/working/labels/val/{name.replace('.jpg','.txt')}"
    )

print("train-val split DONE")
!ls /kaggle/working/images/train | wc -l
!ls /kaggle/working/images/val | wc -l


train-val split DONE
2097
491


In [7]:
from ultralytics import YOLO

# tulis file di terrminal
# %%writefile data.yaml
# path: /kaggle/working
# train: images/train
# val: images/val

# nc: 1
# names:
#   - ball

model = YOLO("yolov8n.pt")

model.train(
    data="/kaggle/working/data.yaml",
    epochs=50,
    imgsz=416,
    batch=16,
    device=0
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, k

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d832415b590>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [10]:
# fine tuning
!yolo detect train \
  model=/kaggle/working/runs/detect/train/weights/best.pt \
  data=/kaggle/working/data.yaml \
  epochs=50 \
  imgsz=640 \
  batch=16 \
  lr0=0.001 \
  patience=20

Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/working/runs/detect/train/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pa

In [12]:
# eexport to onnx
!yolo export \
  model=/kaggle/working/runs/detect/train2/weights/best.pt \
  format=onnx \
  imgsz=640

Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/kaggle/working/runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.9 MB)
requirements: Ultralytics requirements ['onnxslim>=0.1.71', 'onnxruntime'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 225ms
Prepared 4 packages in 232ms
Installed 4 packages in 12ms
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnxruntime==1.23.2
 + onnxslim==0.1.82

requirements: AutoUpdate success ✅ 0.8s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.20.0 opset 22...
/usr/local/lib/python3.12

In [13]:
# export onnx to tensorflow
!yolo export \
  model=/kaggle/working/runs/detect/train2/weights/best.pt \
  format=tf \
  imgsz=640

WARNING ⚠️ Invalid export format='tf', updating to format='tfjs'
Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/kaggle/working/runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.9 MB)
E0000 00:00:1767671005.732260     422 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767671005.784782     422 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767671006.187400     422 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767671006.187451     422 computation_placer.cc:177] computation placer

In [14]:
# export to tflite
!yolo export \
  model=/kaggle/working/runs/detect/train2/weights/best.pt \
  format=tflite \
  imgsz=640

Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/kaggle/working/runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.9 MB)
E0000 00:00:1767671156.694753     556 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767671156.701082     556 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767671156.718725     556 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767671156.718753     556 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the s